In [1]:
import numpy as np
from scipy import optimize
from matplotlib import pyplot as plt

In [67]:
n_bonds = 20#31
vg_raw_data = np.loadtxt(r"data/object_mat.csv", delimiter=",")[:n_bonds, :n_bonds]
c = 2237.404382444853

max_obj = np.sum(vg_raw_data, where=vg_raw_data > 0)
min_obj = np.sum(vg_raw_data, where=vg_raw_data < 0)
penalty = (max_obj - min_obj) * 1.1

# load constrains matrix (A: (num_constr, num_bonds); b: (num_bonds,))
constr_A = np.loadtxt(r"data/constr_A.csv", delimiter=",")[:, :n_bonds]
constr_b = np.loadtxt(r"data/constr_b.csv", delimiter=",")

# C_func_constr = lambda x, vg_data, A, b: vg_ob_qubo(x, vg_data) + c + penalty * np.sum(np.maximum(b - A @ x, 0)**2)

In [79]:
def loss(x, Q=vg_raw_data):
    return x.T @ Q @ x + c + penalty * np.sum(np.maximum(constr_b - constr_A @ x, 0)**2)


cons = {'type':'ineq',
        'fun': lambda x: constr_A @ x - constr_b,
        # 'jac':lambda x: constr_A
        }

opt = {'disp':False}


In [80]:
def bitstring_like(arr, threshold=0.5):
    return ''.join(
        '1' if x > threshold else '0'#('0' if x == 0 else f"{np.round(x, 0)}")
        for x in arr
    )

def hamming_distance(a, b):
    """Calculate the Hamming distance between two bitstrings of equal length."""
    if len(a) != len(b):
        raise ValueError("Bitstrings must be of equal length")
    return sum(x != y for x, y in zip(a, b))

In [81]:
x0 = np.random.rand(n_bonds)    # initial guess

In [82]:
import time

# Methods that support bounds
methods_with_bounds = [
    "L-BFGS-B",
    "TNC",
    "SLSQP",
    "Powell",
    "trust-constr"
]

results = {}

for method in methods_with_bounds:
    try:
        start = time.time()
        res_uncons = optimize.minimize(
            loss, 
            x0, 
            bounds=((0, 1),) * n_bonds,
            method=method,
            options=opt
        )
        elapsed = time.time() - start
        results[method] = {
            "success": res_uncons.success,
            "fun": res_uncons.fun,
            "nfev": res_uncons.nfev,
            "time": elapsed
        }
        print(f"{method:12} | Success: {res_uncons.success} | Fun: {res_uncons.fun:.4f} | NFEV: {res_uncons.nfev} | Time: {elapsed:.4f}s")
        print(f"{method:12} | x = {bitstring_like(res_uncons.x)}")
    except Exception as e:
        print(f"{method:12} | Error: {e}")


C:\Users\xuera\AppData\Local\Temp\ipykernel_27504\3700067659.py:17: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  res_uncons = optimize.minimize(


L-BFGS-B     | Success: True | Fun: 58508.9930 | NFEV: 42 | Time: 0.0030s
L-BFGS-B     | x = 11110111111111111111
TNC          | Success: True | Fun: 58508.9930 | NFEV: 1890 | Time: 0.0634s
TNC          | x = 11110111111111111111
SLSQP        | Success: True | Fun: 1722864.1585 | NFEV: 21 | Time: 0.0010s
SLSQP        | x = 00111111010101011100
Powell       | Success: True | Fun: 58546.6448 | NFEV: 802 | Time: 0.0275s
Powell       | x = 11110111111111111111
trust-constr | Success: False | Fun: 58540.2213 | NFEV: 21000 | Time: 3.2254s
trust-constr | x = 11111011111111111111


## Minimization with constraints

In [83]:
# Methods that support constraints
methods_for_constrs = [
    "COBYLA",
    "COBYQA",
    "TNC",
    "SLSQP",
    "Powell",
    "trust-constr"
]

for method in methods_for_constrs:
    try:
        start = time.time()
        res_uncons = optimize.minimize(
            loss, 
            x0, 
            bounds=((0, 1),) * n_bonds,
            method=method,
            options=opt
        )
        elapsed = time.time() - start
        results[method] = {
            "success": res_uncons.success,
            "fun": res_uncons.fun,
            "nfev": res_uncons.nfev,
            "time": elapsed
        }
        print(f"{method:12} | Success: {res_uncons.success} | Fun: {res_uncons.fun:.4f} | NFEV: {res_uncons.nfev} | Time: {elapsed:.4f}s")
        print(f"{method:12} | x = {bitstring_like(res_uncons.x)}")
    except Exception as e:
        print(f"{method:12} | Error: {e}")


COBYLA       | Success: True | Fun: 58523.7904 | NFEV: 111 | Time: 1.1361s
COBYLA       | x = 11110111111011111111
COBYQA       | Success: True | Fun: 58521.3928 | NFEV: 153 | Time: 0.3220s
COBYQA       | x = 11110011111111111111
TNC          | Success: True | Fun: 58508.9930 | NFEV: 1890 | Time: 0.0475s
TNC          | x = 11110111111111111111
SLSQP        | Success: True | Fun: 1722864.1585 | NFEV: 21 | Time: 0.0000s
SLSQP        | x = 00111111010101011100
Powell       | Success: True | Fun: 58546.6448 | NFEV: 802 | Time: 0.0321s
Powell       | x = 11110111111111111111
trust-constr | Success: False | Fun: 58540.2213 | NFEV: 21000 | Time: 2.6573s
trust-constr | x = 11111011111111111111


## SLSQP only

In [84]:
res_uncons = optimize.minimize(
        loss, 
        x0, 
        bounds=((0, 1),) * n_bonds,
        method='SLSQP',
        options=opt)

In [85]:
res_uncons

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 1722864.158542234
           x: [ 2.403e-02  3.852e-02 ...  1.242e-01  2.012e-01]
         nit: 5
         jac: [-1.690e+05 -2.273e+05 ... -2.277e+05 -2.973e+05]
        nfev: 21
        njev: 1
 multipliers: []

In [86]:
res_cons = optimize.minimize(
    loss,
    x0,
    bounds=((0, 1),) * n_bonds,
    constraints=cons,
    method='SLSQP', 
    options=opt)

In [87]:
res_cons

     message: Positive directional derivative for linesearch
     success: False
      status: 8
         fun: 1722864.158542234
           x: [ 2.403e-02  3.852e-02 ...  1.242e-01  2.012e-01]
         nit: 5
         jac: [-1.690e+05 -2.273e+05 ... -2.277e+05 -2.973e+05]
        nfev: 21
        njev: 1
 multipliers: [ 0.000e+00  2.070e+02 ...  0.000e+00  0.000e+00]

In [88]:
# ref_bitstring = "0111001011111011101011100111101"
ref_bitstring = "11110111111111111111"
print("Scipy.minimize: " + f"d_H={hamming_distance(bitstring_like(res_uncons.x), ref_bitstring):.0f}  " + bitstring_like(res_uncons.x) + " (no constraints)")
print("Scipy.minimize: " + f"d_H={hamming_distance(bitstring_like(res_cons.x), ref_bitstring):.0f}  " + bitstring_like(res_cons.x) + " (with constraints)")
print("Brute-force:           " + ref_bitstring + " (ref)")

Scipy.minimize: d_H=9  00111111010101011100 (no constraints)
Scipy.minimize: d_H=9  00111111010101011100 (with constraints)
Brute-force:           11110111111111111111 (ref)
